# 第50章: ピンホールカメラモデルの基礎

## 📋 この章で学ぶこと

この章を終えると、以下ができるようになります：

- [ ] ピンホールカメラモデルの幾何学的意味を理解する
- [ ] 内部パラメータ行列 K を構成・操作できる
- [ ] 3D点から2D画像座標への射影を実装できる
- [ ] 焦点距離・主点が画像に与える影響を可視化できる

## 🎯 前提知識

この章を学ぶには以下の知識が必要です：

- ✅ 線形代数（行列演算、逆行列）
- ✅ NumPy基礎

⏱️ **推定学習時間**: 90-120分  
📊 **難易度**: ★★☆☆☆（初級〜中級）  
🎓 **カテゴリ**: 理論 + 実践

---

## 🌟 はじめに

**3D Computer Vision** の最も基本的な概念は、「3次元の世界がどのように2次元の画像に映し出されるか」を理解することです。

この章では、最もシンプルなカメラモデルである **ピンホールカメラモデル** を学びます。このモデルは：

- NeRFやGaussian Splattingなどの最新技術の基盤
- `camera_intrinsics`、`extrinsics` といったコードの理解に必須
- 「視点」を数学的に記述するための文法

### 🤔 なぜピンホールカメラ？

```
     3D世界              ピンホール              2D画像
    ┌─────────┐              ●                ┌─────────┐
    │  ★     │              │                │    ★   │
    │    ○   │ ───────────→ │ ───────────→   │  ○     │
    │      △ │              │                │ △      │
    └─────────┘              │                └─────────┘
```

光は直進する。この単純な事実から、3D空間の点と2D画像上の点の関係を数学的に導出できます。

In [ ]:
# ============================================================
# 環境設定
# 必要なライブラリをインポートします
# ============================================================

import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import warnings

# 警告を非表示
warnings.filterwarnings('ignore')

# グラフスタイルの設定
plt.rcParams['figure.figsize'] = (12, 6)

# 日本語フォント設定
import matplotlib.font_manager as fm

def setup_japanese_font():
    """日本語フォントを設定する"""
    japanese_fonts = [
        'Hiragino Sans', 'Hiragino Maru Gothic Pro', 'AppleGothic',
        'Yu Gothic', 'MS Gothic',
        'Noto Sans CJK JP', 'IPAexGothic', 'TakaoPGothic',
    ]
    available_fonts = set(f.name for f in fm.fontManager.ttflist)
    for font in japanese_fonts:
        if font in available_fonts:
            plt.rcParams['font.family'] = font
            plt.rcParams['axes.unicode_minus'] = False
            return font
    return None

font_used = setup_japanese_font()
if font_used:
    print(f"日本語フォント: {font_used}")

# 再現性
np.random.seed(42)

print("✅ ライブラリのインポート完了")

---

## 1. ピンホールカメラの幾何学

### 🤔 ピンホールカメラとは？

ピンホールカメラは、レンズを使わずに小さな穴（ピンホール）だけで像を結ぶカメラです。

**物理的な原理:**
- 光は直進する
- ピンホールを通過した光線だけが像を結ぶ
- 結果として、倒立した実像ができる

### 📊 座標系の定義

```
          Y (上)
          │
          │      Z (前方/奥行き)
          │     /
          │    /
          │   /
          │  /
          │ /
          ●───────── X (右)
        (カメラ中心)
```

**カメラ座標系の慣例:**
- X軸: 右方向
- Y軸: 下方向（画像座標系に合わせる場合）または上方向
- Z軸: カメラの視線方向（前方）

In [ ]:
# ============================================================
# ピンホールカメラの幾何学を可視化
# ============================================================

fig = plt.figure(figsize=(14, 6))

# 左: 2D断面図
ax1 = fig.add_subplot(121)

# カメラ中心
ax1.plot(0, 0, 'ko', markersize=10, label='カメラ中心 (ピンホール)')

# 撮像面 (z = f)
f = 1.0  # 焦点距離
ax1.axvline(x=f, color='blue', linestyle='--', linewidth=2, label=f'撮像面 (z = f = {f})')

# 3D点
Z = 5.0  # 3D点の奥行き
Y = 2.0  # 3D点の高さ
ax1.plot(Z, Y, 'r*', markersize=15, label=f'3D点 P = (Y={Y}, Z={Z})')

# 光線（3D点 → ピンホール）
ax1.plot([0, Z], [0, Y], 'g-', linewidth=2, alpha=0.7)

# 像の位置（類似三角形より）
y_image = f * Y / Z
ax1.plot(f, y_image, 'b^', markersize=12, label=f'像 p = (y = {y_image:.2f})')
ax1.plot([0, f], [0, y_image], 'b-', linewidth=2, alpha=0.7)

# 注釈
ax1.annotate('', xy=(f, 0), xytext=(0, 0),
            arrowprops=dict(arrowstyle='<->', color='purple'))
ax1.text(f/2, -0.3, 'f (焦点距離)', ha='center', fontsize=11, color='purple')

ax1.set_xlabel('Z (奥行き)', fontsize=12)
ax1.set_ylabel('Y (高さ)', fontsize=12)
ax1.set_title('ピンホールカメラの断面図', fontsize=14, fontweight='bold')
ax1.legend(loc='upper left')
ax1.grid(True, alpha=0.3)
ax1.set_xlim(-0.5, 6)
ax1.set_ylim(-1, 3)
ax1.set_aspect('equal')

# 右: 類似三角形の説明
ax2 = fig.add_subplot(122)
ax2.text(0.5, 0.8, '類似三角形による射影', ha='center', fontsize=16, fontweight='bold',
         transform=ax2.transAxes)

ax2.text(0.5, 0.6, r'$\frac{y}{f} = \frac{Y}{Z}$', ha='center', fontsize=24,
         transform=ax2.transAxes)

ax2.text(0.5, 0.4, r'$y = f \cdot \frac{Y}{Z}$', ha='center', fontsize=24,
         transform=ax2.transAxes, color='blue')

ax2.text(0.5, 0.2, '同様に: ' + r'$x = f \cdot \frac{X}{Z}$', ha='center', fontsize=18,
         transform=ax2.transAxes)

ax2.axis('off')

plt.tight_layout()
plt.show()

print(f"💡 数値例:")
print(f"  3D点: P = (X, Y, Z) = (0, {Y}, {Z})")
print(f"  焦点距離: f = {f}")
print(f"  像の位置: y = f * Y / Z = {f} * {Y} / {Z} = {y_image:.2f}")

---

## 2. 内部パラメータ行列 K

### 🤔 内部パラメータとは？

**内部パラメータ（Intrinsic Parameters）** は、カメラ固有の特性を表します：

- **焦点距離** (f_x, f_y): ピクセル単位での焦点距離
- **主点** (c_x, c_y): 光軸と撮像面の交点

### 📊 内部パラメータ行列

$$
K = \begin{bmatrix}
f_x & 0 & c_x \\
0 & f_y & c_y \\
0 & 0 & 1
\end{bmatrix}
$$

**各パラメータの意味:**

| パラメータ | 意味 | 典型的な値 |
|-----------|------|------------|
| f_x, f_y | 焦点距離（ピクセル単位） | 500-2000 |
| c_x | 主点のx座標 | 画像幅/2 |
| c_y | 主点のy座標 | 画像高さ/2 |

In [ ]:
# ============================================================
# 内部パラメータ行列の定義と使用
# ============================================================

def create_intrinsic_matrix(fx, fy, cx, cy):
    """
    内部パラメータ行列を生成
    
    K = | fx  0  cx |
        | 0  fy  cy |
        | 0   0   1 |
    """
    K = np.array([
        [fx,  0, cx],
        [ 0, fy, cy],
        [ 0,  0,  1]
    ], dtype=np.float64)
    return K

# 典型的なカメラパラメータ（640x480の画像を想定）
image_width = 640
image_height = 480

fx = 500  # x方向の焦点距離（ピクセル）
fy = 500  # y方向の焦点距離（ピクセル）
cx = image_width / 2   # 主点 x
cy = image_height / 2  # 主点 y

K = create_intrinsic_matrix(fx, fy, cx, cy)

print("内部パラメータ行列 K:")
print(K)
print()
print(f"画像サイズ: {image_width} x {image_height}")
print(f"焦点距離: fx = {fx}, fy = {fy}")
print(f"主点: (cx, cy) = ({cx}, {cy})")

---

## 3. 3D → 2D 射影の実装

### 📊 射影の流れ

```
3D点（カメラ座標系）→ 正規化画像座標 → ピクセル座標
     (X, Y, Z)      →    (X/Z, Y/Z)   →   (u, v)
```

**射影式:**

$$
\begin{bmatrix} u \\ v \\ 1 \end{bmatrix} = 
\frac{1}{Z} K \begin{bmatrix} X \\ Y \\ Z \end{bmatrix}
$$

展開すると:
$$
u = f_x \frac{X}{Z} + c_x
$$
$$
v = f_y \frac{Y}{Z} + c_y
$$

In [ ]:
# ============================================================
# 3D → 2D 射影の実装
# ============================================================

def project_point(point_3d, K):
    """
    3D点を2D画像座標に射影
    
    Parameters:
    -----------
    point_3d : array (3,)
        カメラ座標系の3D点 (X, Y, Z)
    K : array (3, 3)
        内部パラメータ行列
    
    Returns:
    --------
    point_2d : array (2,)
        ピクセル座標 (u, v)
    """
    X, Y, Z = point_3d
    
    # 正規化画像座標
    x_norm = X / Z
    y_norm = Y / Z
    
    # ピクセル座標に変換
    u = K[0, 0] * x_norm + K[0, 2]  # fx * x + cx
    v = K[1, 1] * y_norm + K[1, 2]  # fy * y + cy
    
    return np.array([u, v])


def project_points_batch(points_3d, K):
    """
    複数の3D点を一度に射影（行列演算版）
    
    Parameters:
    -----------
    points_3d : array (N, 3)
        カメラ座標系の3D点群
    K : array (3, 3)
        内部パラメータ行列
    
    Returns:
    --------
    points_2d : array (N, 2)
        ピクセル座標群
    """
    # 同次座標への射影: p = K @ P
    points_h = (K @ points_3d.T).T  # (N, 3)
    
    # 正規化（Zで割る）
    points_2d = points_h[:, :2] / points_h[:, 2:3]
    
    return points_2d


# テスト
test_point = np.array([1.0, 0.5, 5.0])  # X=1, Y=0.5, Z=5
projected = project_point(test_point, K)

print(f"3D点: {test_point}")
print(f"射影された2D点: {projected}")
print()
print("検算:")
print(f"  u = fx * X/Z + cx = {fx} * {test_point[0]}/{test_point[2]} + {cx} = {fx * test_point[0]/test_point[2] + cx}")
print(f"  v = fy * Y/Z + cy = {fy} * {test_point[1]}/{test_point[2]} + {cy} = {fy * test_point[1]/test_point[2] + cy}")

---

## 4. 3D立方体の射影

射影の理解を深めるために、3D空間の立方体を2D画像に射影してみましょう。

In [ ]:
# ============================================================
# 3D立方体の定義と射影
# ============================================================

def create_cube(center, size):
    """
    立方体の8つの頂点を生成
    
    Parameters:
    -----------
    center : array (3,)
        立方体の中心座標
    size : float
        立方体の一辺の長さ
    
    Returns:
    --------
    vertices : array (8, 3)
        頂点座標
    edges : list of tuples
        辺のインデックスペア
    """
    half = size / 2
    offsets = np.array([
        [-1, -1, -1],
        [+1, -1, -1],
        [+1, +1, -1],
        [-1, +1, -1],
        [-1, -1, +1],
        [+1, -1, +1],
        [+1, +1, +1],
        [-1, +1, +1],
    ]) * half
    
    vertices = center + offsets
    
    # 12本の辺
    edges = [
        (0, 1), (1, 2), (2, 3), (3, 0),  # 前面
        (4, 5), (5, 6), (6, 7), (7, 4),  # 後面
        (0, 4), (1, 5), (2, 6), (3, 7),  # 接続
    ]
    
    return vertices, edges


# 立方体を作成（カメラから5m先、中心が原点から少しずれた位置）
cube_center = np.array([0.5, 0.3, 5.0])
cube_size = 1.0
vertices, edges = create_cube(cube_center, cube_size)

print("立方体の頂点:")
for i, v in enumerate(vertices):
    print(f"  頂点 {i}: {v}")

In [ ]:
# ============================================================
# 立方体の射影と可視化
# ============================================================

# 3D頂点を2Dに射影
vertices_2d = project_points_batch(vertices, K)

fig = plt.figure(figsize=(16, 6))

# 左: 3Dビュー
ax1 = fig.add_subplot(121, projection='3d')

# 立方体の辺を描画
for i, j in edges:
    ax1.plot3D([vertices[i, 0], vertices[j, 0]],
               [vertices[i, 1], vertices[j, 1]],
               [vertices[i, 2], vertices[j, 2]], 'b-', linewidth=2)

# 頂点をマーク
ax1.scatter(vertices[:, 0], vertices[:, 1], vertices[:, 2], 
            c='red', s=50, marker='o')

# カメラ位置（原点）
ax1.scatter([0], [0], [0], c='green', s=100, marker='^', label='カメラ')

ax1.set_xlabel('X')
ax1.set_ylabel('Y')
ax1.set_zlabel('Z')
ax1.set_title('3D空間の立方体', fontsize=14, fontweight='bold')
ax1.legend()

# 右: 2D射影
ax2 = fig.add_subplot(122)

# 画像の境界を描画
ax2.axhline(y=0, color='gray', linestyle='--', alpha=0.5)
ax2.axhline(y=image_height, color='gray', linestyle='--', alpha=0.5)
ax2.axvline(x=0, color='gray', linestyle='--', alpha=0.5)
ax2.axvline(x=image_width, color='gray', linestyle='--', alpha=0.5)

# 主点をマーク
ax2.plot(cx, cy, 'g+', markersize=15, markeredgewidth=2, label='主点')

# 立方体の辺を描画
for i, j in edges:
    ax2.plot([vertices_2d[i, 0], vertices_2d[j, 0]],
             [vertices_2d[i, 1], vertices_2d[j, 1]], 'b-', linewidth=2)

# 頂点をマーク
ax2.scatter(vertices_2d[:, 0], vertices_2d[:, 1], c='red', s=50, marker='o')

ax2.set_xlabel('u (ピクセル)', fontsize=12)
ax2.set_ylabel('v (ピクセル)', fontsize=12)
ax2.set_title('2D画像への射影', fontsize=14, fontweight='bold')
ax2.set_xlim(-50, image_width + 50)
ax2.set_ylim(image_height + 50, -50)  # y軸を反転（画像座標系）
ax2.set_aspect('equal')
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

---

## 5. 焦点距離の影響

焦点距離は「画角（Field of View, FOV）」に直接影響します。

- **焦点距離が長い** → 画角が狭い（望遠）→ 物体が大きく見える
- **焦点距離が短い** → 画角が広い（広角）→ 物体が小さく見える

In [ ]:
# ============================================================
# 焦点距離の影響を可視化
# ============================================================

focal_lengths = [200, 500, 1000, 2000]

fig, axes = plt.subplots(1, 4, figsize=(18, 5))

for ax, f in zip(axes, focal_lengths):
    # 新しい内部パラメータ行列
    K_new = create_intrinsic_matrix(f, f, cx, cy)
    
    # 射影
    vertices_2d_new = project_points_batch(vertices, K_new)
    
    # 画像の境界
    ax.add_patch(plt.Rectangle((0, 0), image_width, image_height, 
                               fill=False, edgecolor='gray', linestyle='--'))
    
    # 立方体の辺
    for i, j in edges:
        ax.plot([vertices_2d_new[i, 0], vertices_2d_new[j, 0]],
                [vertices_2d_new[i, 1], vertices_2d_new[j, 1]], 'b-', linewidth=2)
    
    # 頂点
    ax.scatter(vertices_2d_new[:, 0], vertices_2d_new[:, 1], c='red', s=30)
    
    # 画角を計算
    fov = 2 * np.arctan(image_width / (2 * f)) * 180 / np.pi
    
    ax.set_xlim(-50, image_width + 50)
    ax.set_ylim(image_height + 50, -50)
    ax.set_aspect('equal')
    ax.set_title(f'f = {f}\nFOV = {fov:.1f}°', fontsize=12, fontweight='bold')
    ax.axis('off')

plt.suptitle('焦点距離と画角の関係', fontsize=16, fontweight='bold')
plt.tight_layout()
plt.show()

print("💡 観察ポイント:")
print("  - 焦点距離が長いほど、物体が大きく見える（ズームイン効果）")
print("  - 焦点距離が短いほど、画角が広くなり、物体が小さく見える")

---

## 6. 主点のオフセット

主点 (cx, cy) は通常、画像の中心にありますが、ずれている場合もあります。

In [ ]:
# ============================================================
# 主点のオフセットの影響
# ============================================================

principal_points = [
    (cx, cy),           # 中央
    (cx + 100, cy),     # 右にオフセット
    (cx, cy + 80),      # 下にオフセット
    (cx - 150, cy - 60) # 左上にオフセット
]

fig, axes = plt.subplots(1, 4, figsize=(18, 5))

for ax, (cx_new, cy_new) in zip(axes, principal_points):
    K_new = create_intrinsic_matrix(fx, fy, cx_new, cy_new)
    vertices_2d_new = project_points_batch(vertices, K_new)
    
    # 画像の境界
    ax.add_patch(plt.Rectangle((0, 0), image_width, image_height, 
                               fill=False, edgecolor='gray', linestyle='--'))
    
    # 主点をマーク
    ax.plot(cx_new, cy_new, 'g+', markersize=20, markeredgewidth=3)
    ax.plot(image_width/2, image_height/2, 'k+', markersize=15, markeredgewidth=2, alpha=0.3)
    
    # 立方体
    for i, j in edges:
        ax.plot([vertices_2d_new[i, 0], vertices_2d_new[j, 0]],
                [vertices_2d_new[i, 1], vertices_2d_new[j, 1]], 'b-', linewidth=2)
    ax.scatter(vertices_2d_new[:, 0], vertices_2d_new[:, 1], c='red', s=30)
    
    ax.set_xlim(-50, image_width + 50)
    ax.set_ylim(image_height + 50, -50)
    ax.set_aspect('equal')
    ax.set_title(f'主点: ({cx_new:.0f}, {cy_new:.0f})', fontsize=12, fontweight='bold')
    ax.axis('off')

plt.suptitle('主点のオフセットの影響', fontsize=16, fontweight='bold')
plt.tight_layout()
plt.show()

print("💡 観察ポイント:")
print("  - 主点が移動すると、射影結果全体がシフトする")
print("  - 物体の形状自体は変化しない")

---

## 7. 「ズーム」と「接近」の違い

焦点距離を変えることと、物体に近づくことは異なる効果をもたらします。

In [ ]:
# ============================================================
# ズーム vs 接近 の比較
# ============================================================

fig, axes = plt.subplots(1, 3, figsize=(16, 5))

# 1. 基準画像（f=500, z=5）
K_base = create_intrinsic_matrix(500, 500, cx, cy)
cube_base = create_cube(np.array([0.5, 0.3, 5.0]), 1.0)[0]
v2d_base = project_points_batch(cube_base, K_base)

ax = axes[0]
ax.add_patch(plt.Rectangle((0, 0), image_width, image_height, 
                           fill=False, edgecolor='gray', linestyle='--'))
for i, j in edges:
    ax.plot([v2d_base[i, 0], v2d_base[j, 0]],
            [v2d_base[i, 1], v2d_base[j, 1]], 'b-', linewidth=2)
ax.scatter(v2d_base[:, 0], v2d_base[:, 1], c='red', s=30)
ax.set_xlim(-50, image_width + 50)
ax.set_ylim(image_height + 50, -50)
ax.set_aspect('equal')
ax.set_title('基準\n(f=500, Z=5m)', fontsize=12, fontweight='bold')
ax.axis('off')

# 2. ズーム（焦点距離を2倍）
K_zoom = create_intrinsic_matrix(1000, 1000, cx, cy)
v2d_zoom = project_points_batch(cube_base, K_zoom)

ax = axes[1]
ax.add_patch(plt.Rectangle((0, 0), image_width, image_height, 
                           fill=False, edgecolor='gray', linestyle='--'))
for i, j in edges:
    ax.plot([v2d_zoom[i, 0], v2d_zoom[j, 0]],
            [v2d_zoom[i, 1], v2d_zoom[j, 1]], 'g-', linewidth=2)
ax.scatter(v2d_zoom[:, 0], v2d_zoom[:, 1], c='red', s=30)
ax.set_xlim(-50, image_width + 50)
ax.set_ylim(image_height + 50, -50)
ax.set_aspect('equal')
ax.set_title('ズーム（焦点距離2倍）\n(f=1000, Z=5m)', fontsize=12, fontweight='bold')
ax.axis('off')

# 3. 接近（距離を半分）
cube_close = create_cube(np.array([0.25, 0.15, 2.5]), 1.0)[0]
v2d_close = project_points_batch(cube_close, K_base)

ax = axes[2]
ax.add_patch(plt.Rectangle((0, 0), image_width, image_height, 
                           fill=False, edgecolor='gray', linestyle='--'))
for i, j in edges:
    ax.plot([v2d_close[i, 0], v2d_close[j, 0]],
            [v2d_close[i, 1], v2d_close[j, 1]], 'orange', linewidth=2)
ax.scatter(v2d_close[:, 0], v2d_close[:, 1], c='red', s=30)
ax.set_xlim(-50, image_width + 50)
ax.set_ylim(image_height + 50, -50)
ax.set_aspect('equal')
ax.set_title('接近（距離半分）\n(f=500, Z=2.5m)', fontsize=12, fontweight='bold')
ax.axis('off')

plt.suptitle('ズーム vs 接近', fontsize=16, fontweight='bold')
plt.tight_layout()
plt.show()

print("💡 観察ポイント:")
print("  - ズーム: 物体のサイズは拡大するが、遠近感（パースペクティブ）は変わらない")
print("  - 接近: 物体のサイズが拡大し、遠近感も強調される")
print("  - 接近すると、立方体の前面と後面の大きさの差が大きくなる")

---

## まとめ

### 🎯 このノートブックで学んだこと

**ピンホールカメラの幾何学**
- ✓ 光は直進し、類似三角形により像が形成される
- ✓ 3D点 (X, Y, Z) → 2D点 (u, v) への射影

**内部パラメータ行列 K**
- ✓ 焦点距離 (f_x, f_y): 画角に影響
- ✓ 主点 (c_x, c_y): 画像の中心位置

**射影式**
- ✓ $u = f_x \frac{X}{Z} + c_x$
- ✓ $v = f_y \frac{Y}{Z} + c_y$

### 📊 内部パラメータチートシート

| パラメータ | 記号 | 影響 |
|-----------|------|------|
| 焦点距離 | f_x, f_y | 画角（FOV）、物体の見かけの大きさ |
| 主点 | c_x, c_y | 画像上での中心位置 |
| 画角 | FOV | $2 \arctan\left(\frac{W}{2f}\right)$ |

---

## 🎓 自己評価クイズ

### Q1: 焦点距離を2倍にすると、画角はどうなりますか？

<details>
<summary>💡 答えを見る</summary>

**答え**: 画角は約半分になります。

理由: $\text{FOV} = 2 \arctan\left(\frac{W}{2f}\right)$

焦点距離が大きくなると、$\arctan$の引数が小さくなり、画角が狭くなります。

</details>

---

### Q2: 3D点 (2, 1, 4) が内部パラメータ K = [[500, 0, 320], [0, 500, 240], [0, 0, 1]] で射影されると、2D座標は？

<details>
<summary>💡 答えを見る</summary>

**答え**: (u, v) = (570, 365)

計算:
- $u = f_x \cdot \frac{X}{Z} + c_x = 500 \cdot \frac{2}{4} + 320 = 250 + 320 = 570$
- $v = f_y \cdot \frac{Y}{Z} + c_y = 500 \cdot \frac{1}{4} + 240 = 125 + 240 = 365$

</details>

---

### Q3: 主点が画像の中心からずれていると、どのような影響がありますか？

<details>
<summary>💡 答えを見る</summary>

**答え**: 射影された画像全体がシフトします。

主点 (c_x, c_y) は射影式において定数項として機能するため、すべての点が同じ量だけ平行移動します。物体の形状やパースペクティブには影響しません。

</details>

---

### ✅ 学習チェックリスト

- [ ] ピンホールカメラモデルを図で説明できる
- [ ] 内部パラメータ行列 K を構成できる
- [ ] 3D点を2D座標に射影する計算ができる
- [ ] 焦点距離と画角の関係を説明できる
- [ ] ズームと接近の違いを説明できる

---

**次のステップ**: Notebook 51で、**座標変換とカメラキャリブレーション**を学びます！

外部パラメータ [R|t] を導入し、ワールド座標系からカメラ座標系への変換を理解します。